## <font color='darkblue'><center>Qermit is: 
    * A python package for automatically running error mitigated experiments
    * That is open-source
    * That supports any pytket backend including common hardware providers
    * That makes running error mitigation schemes as easy as running any experiment
    * That makes constructing new error mitigation schemes easy
    * That provides customisable composition of schemes
    

<center><img src="ZNETaskGraph35.png" width=380 height=350 />

In [ ]:
from pytket import Circuit
from pytket.backends import Backend
from pytket.backends.backendresult import BackendResult
from pytket.circuit.display import render_circuit_jupyter

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from pytket.extensions.qiskit.backends.config import set_ibmq_config 
# set the below as appropriate
# set_ibmq_config(ibmq_api_token="",
#                 hub="", 
#                 group="", 
#                 project="")



In [ ]:
from typing import List, Tuple

## <font color='darkblue'><center>MitTask
    * _method -> "run this circuit on this backend"
    * _n_in_wires -> 1
    * _n_out_wires -> 1
    * _label -> "RunCircuits"
    
<center><img src="BasicTG.png" width=200 height=200 />

In [ ]:
from qermit import MitTask, CircuitShots

def run_circuit_task_gen(backend: Backend) -> MitTask:
    """
    For each Circuit passed to the task, runs the Circuit through the Backend
    and returns a BackendResult containing counts

    :param backend: Backend for running Circuits through
    :type backend: Backend
    """

    def task(obj, circuits: List[CircuitShots]) -> Tuple[List[BackendResult]]:
        c, s = map(list, zip(*circuits))
        handles = backend.process_circuits(c, n_shots=s)
        results = backend.get_results(handles)
        return (results,)

    return MitTask(
        _label="RunCircuits", _n_in_wires=1, _n_out_wires=1, _method=task
    )

<center><img src="BasicTG.png" width=130 height=130 />

In [ ]:
c = Circuit(2).CX(0,1).measure_all()
render_circuit_jupyter(c)

In [ ]:
from pytket.extensions.qiskit import IBMQEmulatorBackend

# make backend object
quito_backend = IBMQEmulatorBackend('ibmq_quito')
# make MitTask object
quito_run_circuit_task = run_circuit_task_gen(quito_backend)

# run circuits through MitTask object
circuits_wire = [CircuitShots(c, 20)]
results = quito_run_circuit_task((circuits_wire,))
print(results[0][0].get_counts())

## <font color='navy'><center>TaskGraph
    * _task_graph -> networkx MultiDiGraph representing dependencies between MitTask
    * run -> topological sort on _task_graph nodes, run MitTask callable in order, 
    add data to graph edges
    * append -> Insert MitTask vertex to back of _task_graph
    * prepend -> Insert MitTask vertex to front of _task_graph

In [ ]:
from qermit import TaskGraph

tg = TaskGraph()
tg.get_task_graph()

In [ ]:
tg.append(quito_run_circuit_task)

<center><img src="BasicTG.png" width=100 height=100 />

In [ ]:
circuits_wire = [CircuitShots(c, 20)]
results = tg.run((circuits_wire,))
print(results[0][0].get_counts())

## <font color='navy'><center>MitRes
    * run -> Converts List[CircuitShots] to List[BackendResult]
    * append -> Checks that MitTask passed to append receives and 
    returns List[BackendResult]
    * prepend -> Checks that MitTask passed to prepend receives and 
    returns List[CircuitShots]

In [ ]:
from qermit import MitRes
mitres = MitRes(quito_backend)
mitres.get_task_graph()

In [ ]:
def compile_circuit_task_gen(backend: Backend) -> MitTask:
    """
    For each Circuit passed to the task, compiles the Circuit for the Backend.
    
    :param backend: Backend for running Circuits through
    :type backend: Backend
    """
    def task(obj, circuits: List[CircuitShots]) -> Tuple[List[CircuitShots]]:
        return ([(backend.get_compiled_circuit(cs.Circuit), cs.Shots) for cs in circuits],)

    return MitTask(
        _label="CompileCircuits", _n_in_wires=1, _n_out_wires=1, _method=task
    )

In [ ]:
mitres.prepend(compile_circuit_task_gen(quito_backend))
mitres.get_task_graph()

In [ ]:
c = Circuit(2).H(0).CX(0,1).measure_all()
render_circuit_jupyter(c)

In [ ]:
circuits_wire = [CircuitShots(c, 2000)]
results = mitres.run(circuits_wire)
print(results[0].get_counts())

<span style="color:red">**EXERCISE 1**</span>

Write your own MitTask generator function that allows the [compiler pass](https://github.com/CQCL/pytket/blob/main/examples/compilation_example.ipynb) to be one of the inputs. 

In [ ]:
from qermit.frame_randomisation import gen_Frame_Randomisation_MitRes

fr_mitres = gen_Frame_Randomisation_MitRes(quito_backend, samples = 40)

<center><img src="FrameRandomisationMitRes.png" width=250 height=250 />


In [ ]:
circuits_wire = [CircuitShots(c, 2000)]
results = fr_mitres.run(circuits_wire)
print(results[0].get_counts())

<span style="color:red">**EXERCISE 2**</span>

How does the number of `samples` affect the results and the runtime?

In [ ]:
from qermit.spam import gen_FullyCorrelated_SPAM_MitRes

correlations = [quito_backend.backend_info.architecture.nodes]
spam_mitres = gen_FullyCorrelated_SPAM_MitRes(quito_backend, 
                                              calibration_shots=2000, 
                                              correlations=correlations)
results = spam_mitres.run(circuits_wire)
print(results[0].get_counts())

<center><img src="SPAMMitres.png" width=250 height=250 />

<span style="color:red">**EXERCISE 3**</span>

Try using [uncorrelated SPAM](https://cqcl.github.io/Qermit/spam.html#qermit.spam.spam_mitres.gen_UnCorrelated_SPAM_MitRes) instead, how do the results change?

## <font color='navy'><center>MitEx
    * run -> Converts List[ObservableExperiment] to List[QubitPauliOperator]
    * append -> Checks that MitTask passed to append receives and 
    returns List[QubitPauliOperator]
    * prepend -> Checks that MitTask passed to prepend receives and 
    returns List[ObservableExperiment]

In [ ]:
from qermit import MitEx

mitex = MitEx(quito_backend)

<center><img src="Mitex.png" width=300 height=300 />


In [ ]:
from pytket.circuit import Qubit, PauliExpBox
from pytket.passes import DecomposeBoxes
from pytket.pauli import QubitPauliString, Pauli
from pytket.utils import QubitPauliOperator
from qermit import ObservableTracker, AnsatzCircuit, SymbolsDict, ObservableExperiment

peb_xyz = PauliExpBox([Pauli.X, Pauli.Y, Pauli.Z], 0.25)

c = Circuit(3)
c.add_pauliexpbox(peb_xyz, [Qubit(0), Qubit(1), Qubit(2)]).Rz(0.2, 0).Rz(0.3, 1).Rz(0.4, 2)
DecomposeBoxes().apply(c)

render_circuit_jupyter(c)

In [ ]:
qubit_pauli_string = QubitPauliString([Qubit(0), Qubit(1), Qubit(2)], [Pauli.Z, Pauli.Z, Pauli.Z])
ansatz_circuit = AnsatzCircuit(c, 10000, SymbolsDict())

experiment = [ObservableExperiment(ansatz_circuit, ObservableTracker(QubitPauliOperator({qubit_pauli_string: 1.0})))]

In [ ]:
results = mitex.run(experiment)
print(results[0])

In [ ]:
from qermit.zero_noise_extrapolation import gen_ZNE_MitEx

zne_mitex = gen_ZNE_MitEx(quito_backend, [3,5], show_fit = True)

<center><img src="ZNETaskGraph35.png" width=300 height=300 />

In [ ]:
results = zne_mitex.run(experiment)
print(results[0])

<span style="color:red">**EXERCISE 4**</span>

Try extending the fold to 7. What is the best tradeoff between number of points and prediction?

## <font color='navy'><center>Combining Schemes
    * TaskGraphs for running error mitigation schemes are constructed on call
    * Each construction builds around a MitRes or MitEx object
    * Via optional arguments, a custom MitRes or MitEx object can be passed to 
    the generator to construct a TaskGraph around
    * MitRes schemes can be built around MitRes objects, MitEx schemes can be 
    built around MitRes or MitEx objects

In [ ]:
spam_fr_mitres = gen_FullyCorrelated_SPAM_MitRes(quito_backend, 
                                              calibration_shots=2000, 
                                              correlations=correlations,
                                              correction_mitres=fr_mitres)
results = spam_fr_mitres.run(circuits_wire)
print(results[0].get_counts())

<center><img src="SPAMFRMitres.png" width=265 height=220 />

In [ ]:
spam_fr_mitex = MitEx(quito_backend, mitres=spam_fr_mitres)
results = spam_fr_mitex.run(experiment)
print(results[0])

<center><img src="SPAMFRMitex.png" width=300 height=300 />

In [ ]:
zne_spam_fr_mitex = gen_ZNE_MitEx(quito_backend, 
                                  [3,5], 
                                  experiment_mitex=spam_fr_mitex,
                                  show_fit=True)
results = zne_spam_fr_mitex.run(experiment)
print(results[0])

<span style="color:red">**EXERCISE 5**</span>

Can we change the ordering of the mitigation methods applied? What are the different costs/benefits of these alternative orderings?

## <font color='navy'><center>Comparison for ibmq_quito noise model
    * Actual value -> 1.0
    * MitEx -> 0.867
    * MitEx with frame randomisation and SPAM -> 0.951
    * MitEx with frame randomisation, SPAM and ZNE -> 0.998

## <font color='navy'><center>Comparison for ibm_perth device
    * Actual value -> 1.0
    * MitEx -> 0.813
    * MitEx with frame randomisation and SPAM -> 0.986
    * MitEx with frame randomisation, SPAM and ZNE -> 1.041

<span style="color:red">**EXERCISE 6**</span>

Verify these results yourself using the ibm_perth device, by copying and modifying the code above.

<span style="color:red">**EXERCISE 7**</span>

Construct your own Qermit error mitigated experiment for the QAOA algorithm presented in [QAOA_notebook.ipynb](notebooks/QAOA_notebook.ipynb)

# <font color='navy'><center>Thanks!</font>
    
## <center>Volumetric Benchmarking of Error Mitigation with Qermit -> arXiv:2204.09725
## <a href="qerm.it"><font color = 'blue'><center>qerm.it</font></a>